Importing all the required libraries

In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


Reading the CSV File aand breaking the DateTime to seasons, weekday

In [7]:
df = pd.read_csv("/content/sample_data/Part1_Crime_data.csv")
df.shape
# print(df['CrimeDateTime'])
#df['CrimeDateTime'] = df['CrimeDateTime'].astype('datetime64[ns]')
df['CrimeDateTime'] = df['CrimeDateTime'].astype('datetime64[ns]')
df['Crime_Year']=df.CrimeDateTime.dt.year
df['Crime_Month']=df.CrimeDateTime.dt.month_name()
df['Crime_Day']=df.CrimeDateTime.dt.day
df['Crime_Hour']=df.CrimeDateTime.dt.hour
df['Crime_WeekDay'] = df.CrimeDateTime.dt.weekday
print(df['Crime_WeekDay'])


0         5
1         5
2         5
3         5
4         5
         ..
434539    2
434540    2
434541    2
434542    2
434543    2
Name: Crime_WeekDay, Length: 434544, dtype: int64


Categorizing the Crime types into Low Medium and High (0,1,2)

In [8]:
replace_val = "0"
df.replace("LARCENY", replace_val, inplace=True)
df.replace("LARCENY FROM AUTO", replace_val, inplace=True)
df.replace("AUTO THEFT", replace_val, inplace=True)
df.replace("BURGLARY", replace_val, inplace=True)

replace_val = "1"
df.replace("ROBBERY", replace_val, inplace=True)
df.replace("ROBBERY - STREET", replace_val, inplace=True)
df.replace("ROBBERY - COMMERCIAL", replace_val, inplace=True)
df.replace("ROBBERY - RESIDENCE", replace_val, inplace=True)
df.replace("ROBBERY - CARJACKING", replace_val, inplace=True)
df.replace("ASSAULT", replace_val, inplace=True)
df.replace("COMMON ASSAULT", replace_val, inplace=True)
df.replace("AGG. ASSAULT", replace_val, inplace=True)

#Fatal
replace_val = "2"
df.replace("SHOOTING", replace_val, inplace=True)
df.replace("RAPE", replace_val, inplace=True)
df.replace("ARSON", replace_val, inplace=True)
df.replace("HOMICIDE", replace_val, inplace=True)


Categorizing months into seasons

In [9]:

#changing the labels of month to quantitative values
#1-Spring
#2-Summer
#3-Rainy
#4-Winter
df.replace("January", 4, inplace=True)
df.replace("February", 4, inplace=True)
df.replace("March", 1, inplace=True)
df.replace("April", 1, inplace=True)
df.replace("May", 1, inplace=True)
df.replace("June", 2, inplace=True)
df.replace("July", 2, inplace=True)
df.replace("August", 2, inplace=True)
df.replace("September", 3, inplace=True)
df.replace("October", 3, inplace=True)
df.replace("November", 3, inplace=True)
df.replace("December", 4, inplace=True)



#Creating a range of dataframe fro weekday
#0 - Weekday
#1 - Weekend
df['Crime_WeekDay'] = np.where((df['Crime_WeekDay']) < 5,0,1)

In [10]:
df = df.rename(columns={'Description': 'Level', 'Crime_Month': 'Season'})
count = df.groupby('Level').size()
col_list = ['Season','Crime_Hour','District','Neighborhood','Crime_WeekDay']

df2 = df[col_list]
print(count)

Level
0    251220
1    170194
2     13130
dtype: int64


Extracting a sample of 1% size for Low

In [12]:
our_condition = df['Level'] == '0'
 
#Retirive the index
index = our_condition[our_condition == True].index
 
#sample based on condition 
low = df[our_condition].sample(frac = 0.01)
 
#output 
low

,RowID,CrimeDateTime,CrimeCode,Location,Level,Inside_Outside,Weapon,Post,District,Neighborhood,...,Longitude,GeoLocation,Premise,VRIName,Total_Incidents,Crime_Year,Season,Crime_Day,Crime_Hour,Crime_WeekDay
401887,401888,2013-05-04 11:00:00,7A,900 HARLEM AVE,0,O,NaN,713,WESTERN,HARLEM PARK,...,-76.6333,"(39.2969,-76.6333)",STREET,Central,1,2013,1,4,11,1
2111,2112,2022-02-09 17:10:00,6F,3700 Toone St,0,NaN,NaN,231,SOUTHEAST,CANTON INDUSTRIAL AREA,...,-76.5656,"(39.2786,-76.5656)",NaN,NaN,1,2022,4,9,17,0
39791,39792,2021-02-02 02:05:00,5F,5400 HARFORD RD,0,I,NaN,423,NORTHEAST,WALTHERSON,...,-76.5624,"(39.351,-76.5624)",RETAIL/SMALL BUSINES,NaN,1,2021,4,2,2,0
301917,301918,2015-06-22 07:30:00,5A,400 E LORRAINE AVE,0,I,NaN,513,NORTHERN,HARWOOD,...,-76.6110,"(39.32,-76.611)",ROW/TOWNHOUSE-OCC,NaN,1,2015,2,22,7,0
184339,184340,2017-10-17 12:00:00,5C,2900 ORLEANS ST,0,I,NaN,221,SOUTHEAST,ELLWOOD PARK/MONUMENT,...,-76.5764,"(39.2959,-76.5764)",ROW/TOWNHOUSE-OCC,NaN,1,2017,3,17,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279018,279019,2015-11-30 20:30:00,6G,200 E PRATT ST,0,I,NaN,112,CENTRAL,INNER HARBOR,...,-76.6114,"(39.2865,-76.6114)",RESTAURANT,NaN,1,2015,3,30,20,0
31875,31876,2021-05-06 19:50:00,6D,300 CHASE ST,0,NaN,NaN,313,EASTERN,JOHNSTON SQUARE,...,-76.6104,"(39.3027,-76.6104)",NaN,NaN,1,2021,1,6,19,0
312039,312040,2015-04-13 08:10:00,7A,1700 GORSUCH AVE,0,O,NaN,411,NORTHEAST,COLDSTREAM HOMESTEAD MONTEBELLO,...,-76.5941,"(39.321,-76.5941)",STREET,NaN,1,2015,1,13,8,0
310304,310305,2015-04-27 09:00:00,7A,2400 LIBERTY HGTS AVE,0,O,NaN,731,WESTERN,MONDAWMIN,...,-76.6540,"(39.3186,-76.654)",PARKING LOT-OUTSIDE,NaN,1,2015,1,27,9,0


Extracting the Sample of size 1% from Medium

In [13]:
our_condition = df['Level'] == '1'
 
#Retirive the index
index = our_condition[our_condition == True].index
 
#sample based on condition 
medium = df[our_condition].sample(frac = 0.01)
 
#output 
medium

,RowID,CrimeDateTime,CrimeCode,Location,Level,Inside_Outside,Weapon,Post,District,Neighborhood,...,Longitude,GeoLocation,Premise,VRIName,Total_Incidents,Crime_Year,Season,Crime_Day,Crime_Hour,Crime_WeekDay
70196,70197,2020-03-20 04:01:00,4A,300 KEY HWY,1,NaN,FIREARM,112,CENTRAL,INNER HARBOR,...,-76.6098,"(39.2808,-76.6098)",NaN,NaN,1,2020,1,20,4,0
332959,332960,2014-10-12 00:30:00,3JF,6600 VINCENT LN,1,I,FIREARM,631,NORTHWEST,REISTERSTOWN STATION,...,-76.7084,"(39.3549,-76.7084)",APT/CONDO - OCCUPIED,NaN,1,2014,3,12,0,1
317047,317048,2015-02-26 20:11:00,3AK,500 CHESTNUT HILL AVE,1,O,KNIFE,525,NORTHERN,WAVERLY,...,-76.6094,"(39.3345,-76.6094)",STREET,NaN,1,2015,4,26,20,0
83355,83356,2019-11-20 16:10:00,4E,5400 WABASH AVE,1,I,NaN,634,NORTHWEST,WOODMERE,...,-76.6919,"(39.3429,-76.6919)",ROW/TOWNHOUSE-OCC,NaN,1,2019,3,20,16,0
411675,411676,2013-02-11 17:15:00,4B,800 W 36TH ST,1,O,KNIFE,531,NORTHERN,HAMPDEN,...,-76.6295,"(39.3313,-76.6295)",STREET,NaN,1,2013,4,11,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240941,240942,2016-09-22 12:55:00,4B,600 MOSHER ST,1,O,KNIFE,123,CENTRAL,UPTON,...,-76.6342,"(39.3009,-76.6342)",STREET,Central,1,2016,3,22,12,0
1293,1294,2022-02-18 06:40:00,4E,900 S CATON AVE,1,NaN,NaN,832,SOUTHWEST,VIOLETVILLE,...,-76.6715,"(39.2738,-76.6715)",NaN,NaN,1,2022,4,18,6,0
44641,44642,2020-12-06 09:00:00,4C,2400 PENNSYLVANIA AVE,1,O,OTHER,733,WESTERN,PENN NORTH,...,-76.6417,"(39.3093,-76.6417)",STREET,Western,1,2020,4,6,9,1
375692,375693,2013-11-04 16:45:00,3B,6300 REISTERSTOWN RD,1,O,NaN,631,NORTHWEST,REISTERSTOWN STATION,...,-76.6999,"(39.3544,-76.6999)",STREET,NaN,1,2013,3,4,16,0


Extracting a sample of size 1% for the level High

In [14]:
high_condition = df['Level'] == '2'
 
#Retirive the index
index = high_condition[high_condition == True].index
 
#sample based on condition 
high = df[high_condition].sample(frac = 0.01)
 
#output 
high

,RowID,CrimeDateTime,CrimeCode,Location,Level,Inside_Outside,Weapon,Post,District,Neighborhood,...,Longitude,GeoLocation,Premise,VRIName,Total_Incidents,Crime_Year,Season,Crime_Day,Crime_Hour,Crime_WeekDay
17329,17330,2021-09-16 18:04:00,9S,2200 BRYANT AVE,2,Outside,FIREARM,731,WESTERN,PARKVIEW/WOODBROOK,...,-76.6497,"(39.3162,-76.6497)",STREET,Western,1,2021,3,16,18,0
393224,393225,2013-07-05 15:21:00,1A,1000 PENNSYLVANIA AV,2,Inside,FIREARM,123,CENTRAL,UPTON,...,-76.6279,"(39.2981,-76.6279)",DWELLING,Central,1,2013,2,5,15,0
214304,214305,2017-03-30 00:00:00,9S,1800 CLIFTON AV,2,Outside,FIREARM,731,WESTERN,MONDAWMIN,...,-76.6467,"(39.3121,-76.6467)",DWELLING,Western,1,2017,1,30,0,0
78014,78015,2020-01-07 20:10:00,9S,5500 LYNVIEW AV,2,Inside,FIREARM,634,NORTHWEST,WOODMERE,...,-76.6928,"(39.3466,-76.6928)",DWELLING,NaN,1,2020,4,7,20,0
184373,184374,2017-10-17 19:53:00,9S,2400 BARCLAY ST,2,Outside,FIREARM,311,EASTERN,BARCLAY,...,-76.6113,"(39.3166,-76.6113)",VEHICLE,NaN,1,2017,3,17,19,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43006,43007,2020-12-24 18:30:00,9S,2900 W BELVEDERE AV,2,Outside,FIREARM,633,NORTHWEST,PIMLICO GOOD NEIGHBORS,...,-76.6712,"(39.3516,-76.6712)",STREET,NaN,1,2020,4,24,18,0
190043,190044,2017-09-11 03:40:00,8AO,5500 MIDWOOD AVE,2,I,FIRE,523,NORTHERN,CAMERON VILLAGE,...,-76.6004,"(39.357,-76.6004)",ROW/TOWNHOUSE-OCC,NaN,1,2017,3,11,3,0
116719,116720,2019-03-17 02:00:00,2B,400 N CURLEY ST,2,I,OTHER,224,SOUTHEAST,ELLWOOD PARK/MONUMENT,...,-76.5757,"(39.2963,-76.5757)",ROW/TOWNHOUSE-OCC,NaN,1,2019,1,17,2,1
183100,183101,2017-10-25 12:29:00,9S,4700 LIBERTY HEIGHTS AV,2,Outside,FIREARM,622,NORTHWEST,HOWARD PARK,...,-76.6958,"(39.3315,-76.6958)",STREET,NaN,1,2017,3,25,12,0


Concatenating the created samples

In [15]:
finalSample = pd.concat([low, medium, high])
# y=df['Description']
# X=df2

Factorizing the Variables

In [16]:


finalSample2 = finalSample[col_list]


#factorize Crime_WeekDay:
weekday_var = pd.factorize(finalSample2['Crime_WeekDay'])
finalSample2['Crime_WeekDay'] = weekday_var[0]
weekday = weekday_var[1] 

#factorize Crime_Month:
month_var = pd.factorize(finalSample2['Season'])
finalSample2['Season'] = month_var[0]
month = month_var[1] 

#factorize Crime_Hour:
hour_var = pd.factorize(finalSample2['Crime_Hour'])
finalSample2['Crime_Hour'] = hour_var[0]
hour = hour_var[1]

#factorize District:
district_var = pd.factorize(finalSample2['District'])
finalSample2['District'] = district_var[0]
district = district_var[1] 

#factorize Neighbourhood:
neighbourhood_var = pd.factorize(finalSample2['Neighborhood'])
finalSample2['Neighborhood'] = neighbourhood_var[0]
neighborhood = neighbourhood_var[1]


y = finalSample['Level']
X = finalSample2

X.head()



<ipython-input-16-6ebbf9c7257a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalSample2['Crime_WeekDay'] = weekday_var[0]
<ipython-input-16-6ebbf9c7257a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalSample2['Season'] = month_var[0]
<ipython-input-16-6ebbf9c7257a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,Season,Crime_Hour,District,Neighborhood,Crime_WeekDay
401887,0,0,0,0,0
2111,1,1,1,1,1
39791,1,2,2,2,1
301917,2,3,3,3,1
184339,3,4,1,4,1


Creating Train and Test data

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state = 123, shuffle = True)

Creating a Model using KNN classifier and GridSearchCV to perform the prediction

In [18]:
knn = KNeighborsClassifier()
from sklearn.model_selection import GridSearchCV
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
  
# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
  
# fitting the model for grid search
grid_search=grid.fit(x_train, y_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


In [19]:
print(grid_search.best_params_)
accuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )



{'n_neighbors': 30}
Accuracy for our training dataset with tuning is : 55.59%


In [20]:
print(classification_report(y_test, grid_search.predict(x_test)))

              precision    recall  f1-score   support

           0       0.61      0.79      0.69       912
           1       0.39      0.23      0.29       571
           2       0.00      0.00      0.00        38

    accuracy                           0.56      1521
   macro avg       0.33      0.34      0.33      1521
weighted avg       0.51      0.56      0.52      1521



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
